In [4]:
import pandas as pd
import numpy as np
import random
import optuna
import sys, os
from datetime import datetime, timezone
from catboost import Pool, cv
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
import polars as pl

# data
is_dshub = os.getenv("NB_USER", False)
if is_dshub:
    train_path = "../../data/train.csv"
    test_path = "../../data/test.csv"
else:
    train_path = "/home/manpm/Developers/kaggle/data/mushrooms/train.csv"
    test_path = "/home/manpm/Developers/kaggle/data/mushrooms/test.csv"
# helpers
sys.path.append("..")
from helpers.loss_functions import *

SEED = 108
random.seed(SEED)
N_FOLDS = 10

train = pl.read_csv(train_path).to_pandas()

In [5]:
skf = StratifiedKFold(n_splits=N_FOLDS)

y_preds = []
y_trues = []
# X = X_train.to_numpy()
for i, (train_index, test_index) in enumerate(
    skf.split(train, train["class"], groups=train["id"])
):
    train.loc[test_index, ["fold"]] = i
train["fold"].unique()

/home/manpm/miniconda3/envs/kaggle/lib/python3.11/site-packages/sklearn/model_selection/_split.py:848: UserWarning: The groups parameter is ignored by StratifiedKFold
  warnings.warn(


,id,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,...,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season,fold
0,0,e,8.80,f,s,u,f,a,c,w,...,None,w,None,None,f,f,None,d,a,0.0
1,1,p,4.51,x,h,o,f,a,c,n,...,y,o,None,None,t,z,None,d,w,0.0
2,2,e,6.94,f,s,b,f,x,c,w,...,s,n,None,None,f,f,None,l,w,0.0
3,3,e,3.88,f,y,g,f,s,None,g,...,None,w,None,None,f,f,None,d,u,0.0
4,4,e,5.85,x,l,w,f,d,None,w,...,None,w,None,None,f,f,None,g,a,0.0


In [6]:
train.to_parquet("../train_fold.parquet")